In [4]:
#--- RNA-seq Data Analysis Tutorial
# By: Ashok K. Sharma, Date: 3rd Oct, 2021

#-- Packages needs to be installed before Data Processing
#install.packages ("pheatmap")
#install.packages ("NMF")
#install.packages("pgirmess") #-- There is a problem in the installation of This packages
#install.packages ("ggrepel")
#install.packages ("RColorBrewer")

#install.packages('BiocManager')
#library (BiocManager)
#BiocManager::install('DESeq2')
#BiocManager::install('EnhancedVolcano')
#BiocManager::install("variancePartition")


In [1]:
library (pheatmap)
library (ggplot2)
library (NMF)
library (variancePartition)
library (EnhancedVolcano)
library (ggrepel)
library (RColorBrewer)
library (DESeq2)
#library (pgirmess)

In [2]:
#packageVersion("Rcpp")
list.files () #Look what files are there in the Main directory

In [1]:
#--- Load All three files: 1. Count Data, 2. Metadata and 3. Length of genes
Gene_counts <- read.csv(file = "GSE81266-expression.txt.txt", sep = "\t", row.names = 1, header = T)
metadata <- read.csv(file = "GSE81266-metadata.txt.txt", sep = "\t", row.names = 1, header = T)
Gene_length <- read.csv(file = "GSE81266-genelength.txt.txt", sep = "\t", header = T)

In [5]:
ls () #Look what files are Loaded in R

In [6]:
head(Gene_counts)
head (metadata)
head (Gene_length)
# Sort by vector name [z]
#dataframe[with(dataframe, order(z)),]

In [7]:
#Check the sturcture of Data: Obervations will be in the Row and Variables will be in the columns
str (Gene_counts)
str (metadata)

In [8]:
#-- Make sure Samples names are in the same order
colnames(Gene_counts)
row.names(metadata)

In [9]:
#--- Counts number of reads mapped to Genes in Each sample
#colSums (Gene_counts)
depth <- colSums(Gene_counts)
#depth <- data.frame (depth)
head (depth)
barplot(depth, las=2, cex.names=.5)

In [10]:
#---- Check the differences in the expression of any one Genes among Healthy and Pouchitis
Gene_counts_t <- data.frame(t(Gene_counts))
row.names (Gene_counts_t)
boxplot (Gene_counts_t$A1BG ~ metadata$prognosis)

In [2]:
#- These are Raw counts which shows expression seems little Higher in Helahty in comparision to FAP and Pouchitis
#-- Now we will try to Normalize this data (TPM Counts): Why this is important?

#Divide the read counts by the length of each gene in kilobases. This gives you reads per kilobase (RPK).
#Count up all the RPK values in a sample and divide this number by 1,000,000. This is your “per million” scaling factor.
#Divide the RPK values by the “per million” scaling factor. This gives you TPM.

r_tpm <- function(dfr,len)
{
  dfr1 <- sweep(dfr,MARGIN=1,(len/10^4),`/`)
  scf <- colSums(dfr1)/(10^6)
  return(sweep(dfr1,2,scf,`/`))
}

In [3]:
Gene_counts_TPM <- r_tpm(Gene_counts, Gene_length$gene_length)

In [11]:
colSums(Gene_counts) # Check counts of reads mapped to genes in each sample
colSums(Gene_counts_TPM) # Check normalized counts of reads mapped to genes in each sample

In [5]:
#---- Check the differences in the expression of any one Genes among Healthy and Pouchitis
Gene_counts_TPM_t <- data.frame(t(Gene_counts_TPM))
row.names (Gene_counts_TPM_t)
boxplot (Gene_counts_TPM_t$A1BG ~ metadata$prognosis)

In [13]:
#--- See the Effect of Normalization and Why it is Important;
boxplot (Gene_counts_t$A1BG ~ metadata$prognosis)
boxplot (Gene_counts_TPM_t$A1BG ~ metadata$prognosis)

In [15]:
#--To make these colors little more beautifiul with colours and expression of this gene in Each sample:
Color <- c("blue", "darkgreen", "darkred")
boxplot (Gene_counts_t$A1BG ~ metadata$prognosis, col=Color, ylab="Gene counts") +
stripchart(Gene_counts_t$A1BG ~ metadata$prognosis, vertical = TRUE, method = "jitter", add = TRUE, pch = 20, col = 'black')

boxplot (Gene_counts_TPM_t$A1BG ~ metadata$prognosis, col=Color, ylab="Gene counts normalized  (TPM)") +
stripchart(Gene_counts_TPM_t$A1BG ~ metadata$prognosis, vertical = TRUE, method = "jitter", add = TRUE, pch = 20, col = 'black')

In [35]:
#---- Save the Normalized TPM Counts If you would like to use them in Future
write.table(Gene_counts_TPM, file="Gene_counts_TPM.txt", sep = "\t")

In [16]:
list.files () #Look what files are there in the Main directory

In [17]:
# Don't forget to Check the row.names should be in the same Order
row.names (metadata)
row.names (Gene_counts_t)
row.names (Gene_counts_TPM_t)

In [18]:
#----- Data Analysis ----
#- Now you know the data structure. And how to play with one gene at a time. Let's start doing little bit more complicated analysis

################
##-- Clustering
################
dim(Gene_counts_TPM)
rn<-nrow(Gene_counts_TPM)
m<-unique(sample(rn, 1000))
rand_data<-Gene_counts_TPM[m,]
d <- dist(t(rand_data), method = "euclidean")

#for(i in 1:9999){
#  m<-unique(sample(rn, 1000))
#  rand_data<-Gene_counts_TPM[m,]
#  d <-d+ dist(t(rand_data), method = "euclidean")	
  
#}
#d1<-d/100000

fit <- hclust(sqrt(d), method="ward.D2") 
plot(fit)


In [17]:
######################################
#---- DESeq - Differential Expression
######################################
# For this analysis we will not use TPM we will use only Counts; Because DESeq will do its own normalization
library (DESeq2)
dim(Gene_counts)
Gene_counts1<-round(Gene_counts)

Gene_counts_Factor <- DESeqDataSetFromMatrix(Gene_counts1, colData=metadata,design= ~prognosis)
Gene_counts_Factor <- DESeq(Gene_counts_Factor)

Gene_counts_Factor_vsd <- varianceStabilizingTransformation(Gene_counts_Factor,blind=FALSE)
#write.csv(assay(Gene_counts_Factor_vsd),file="Outputs/Gene_counts_Factor_vsd_DESEq2.csv")

#--- Different Way to Save VSDs
Gene_counts_Factor_vsd_df <- data.frame(assay(Gene_counts_Factor_vsd))
#write.table(Gene_counts_Factor_vsd_df, file="Outputs/Gene_counts_Factor_vsd_DESEq2.txt", sep = "\t")


[1] 35238    77

converting counts to integer mode
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing
-- replacing outliers and refitting for 107 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)
estimating dispersions
fitting model and testing


In [19]:
head (Gene_counts_Factor_vsd_df)
#-- Transform the VSD counts to make BoxPLOT

Gene_counts_Factor_vsd_df_t <- data.frame (t(Gene_counts_Factor_vsd_df))

#--- Check if VSD normalization is giving similar pattern to TPM?
boxplot (Gene_counts_Factor_vsd_df_t$A1BG ~ metadata$prognosis, col=Color, ylab="Gene counts normalized  (VSD)") +
stripchart(Gene_counts_Factor_vsd_df_t$A1BG ~ metadata$prognosis, vertical = TRUE, method = "jitter", add = TRUE, pch = 20, col = 'black')


In [69]:
#---- Optional Step
#--- If Needed then Merge the Count data with Metadata to Filter specific columns
Genes_vsd_meta <- merge(metadata, Gene_counts_Factor_vsd_df_t, by=0, all=F)
rownames(Genes_vsd_meta) <- Genes_vsd_meta$Row.names; Genes_vsd_meta$Row.names <- NULL

In [12]:
head (Gene_counts_Factor_vsd_df_t)
head (Genes_vsd_meta)

In [18]:
#---------------------------------
#-- Differentially Expressed Genes
#---------------------------------
#Gene_counts_Factor_diff <- results(Gene_counts_Factor,contrast=c("prognosis", "Pouchitis", "Healthy"),filterFun=ihw)
#--Other method
Gene_counts_Factor_diff = results(Gene_counts_Factor, contrast=c("prognosis", "Pouchitis", "Healthy"), alpha = 0.05, pAdjustMethod = "BH")
#write.table(Gene_counts_Factor_diff,file = "Results_DifferentialExpression.txt",row.names = TRUE,col.names = NA,append = FALSE, quote = FALSE, sep = "\t",eol = "\n", na = "NA", dec = ".")


In [20]:
Gene_counts_Factor_diff
Gene_counts_Factor_diff = Gene_counts_Factor_diff[order(Gene_counts_Factor_diff$pvalue),]
summary(Gene_counts_Factor_diff)


In [21]:
#-- Sort based on Log2 Fold Changes
Gene_counts_Factor_diff[with(Gene_counts_Factor_diff, order(log2FoldChange)),]
#-- Sort based on pvalue
Gene_counts_Factor_diff[with(Gene_counts_Factor_diff, order(pvalue)),]


In [22]:
#-- Now Check the Expression of Gene which is Highly Expressed in Healthy RP11-1070B7.2 (Replace - with . in Gname)
boxplot (Gene_counts_Factor_vsd_df_t$IGHGP ~ metadata$prognosis, col=Color, ylab="Gene counts normalized  (VSD)") +
stripchart(Gene_counts_Factor_vsd_df_t$IGHGP ~ metadata$prognosis, vertical = TRUE, method = "jitter", add = TRUE, pch = 20, col = 'black')

In [23]:
#--- Now this is a very Tidious and Time Consuming task to Look for Each Gene. 
# So Let's do something Quick to Get all the Genes which with high fold changes and Significance
#-- If you remember we have stored Differential Expressed Genes Result in ***Gene_counts_Factor_diff*** file 
#-- If you forget What comparisons you have made. Then look at the components of This File

#library (EnhancedVolcano)
EnhancedVolcano(Gene_counts_Factor_diff,
    lab = rownames(Gene_counts_Factor_diff),
    x = 'log2FoldChange',
    y = 'pvalue')

  EnhancedVolcano(Gene_counts_Factor_diff,
   lab = rownames(Gene_counts_Factor_diff),
    x = 'log2FoldChange',
    y = 'pvalue',
    title = 'Pouchitis vs Healthy ',
    pCutoff = 0.05,
    FCcutoff = 0.5,
    pointSize = 3.0,
    labSize = 6.0)

In [24]:
#############################################
#--Downstream analysis such as HEATMAP, PCA
############################################
#-- This analysis will be performed on VSD counts
#-- Randomly select VSD counts of 500 genes 

dim(Gene_counts_Factor_vsd_df)
rn<-nrow(Gene_counts_Factor_vsd_df)
m<-unique(sample(rn, 500))
rand_data<-Gene_counts_Factor_vsd_df[m,]
head (rand_data)
str (rand_data)

In [25]:
#-- Get epithelial cell proportions From The deconvolution file and Add this in the Metadata
cell_proportions <- read.csv(file = "Predicted_cell_type_Composition.tsv", sep = "\t", header = T, row.names = 1)
epithelial_cell <- cell_proportions$Epithelial_cell
epithelial_cell <- data.frame(epithelial_cell)
#--- Prepare a New Metadata to Add Epithelial Cells
metadata_new <- cbind(metadata, epithelial_cell)
rownames(metadata_new)<- rownames(metadata_new)
head (metadata_new)

In [26]:
#--- Before generating HeatMap again check the row.names in new metadata and columnnames in rand_data are in the same order
row.names (metadata_new)
colnames (rand_data)

In [27]:
#################################
#---- Generate SampleWise HeatMap
##################################
library(RColorBrewer)
library("pheatmap")
metadata_filtered <- metadata_new[,c(2,5,7:9)]

#--- Sample data using only random 500 genes
sampleDists <- dist(t(rand_data))
sampleDistMatrix <- as.matrix(sampleDists)
rownames(sampleDistMatrix) <- rownames(metadata_filtered)
colnames(sampleDistMatrix) <- rownames(metadata_filtered)
colors <- colorRampPalette( rev(brewer.pal(9, "Blues")) )(255)

thisheat <- pheatmap(sampleDistMatrix,
                     clustering_distance_rows=sampleDists,
                     clustering_distance_cols=sampleDists,
                     col=colors, annotation_row= metadata_filtered, show_rownames =FALSE,show_colnames = TRUE,legend = TRUE,legend.cex = .05)


ERROR: Error in parse(text = x, srcfile = src): <text>:16:58: unexpected ','
15: #thisheat <- pheatmap(sampleDistMatrix,
16:                      clustering_distance_rows=sampleDists,
                                                             ^


In [28]:
###################
#---- PCA Analysis
###################
pcarip <- prcomp(t(rand_data))

#percent variance for each component
percentVar <- round(100*pcarip$sdev^2/sum(pcarip$sdev^2))

#retrieve the samples coordinates, and the loadings for each gene in each component
aloadrip <- abs(pcarip$rotation)
ripaloadrelative <- sweep(aloadrip, 2, colSums(aloadrip), "/")
#create data frame for plotting
pcaALL <- pcarip$x
pcaR<- data.frame(pcaALL[,1:2],metadata_filtered)

#plot with sample labels
library(ggrepel)
#jpeg("Outputs/pca.jpg", height = 7, width = 7, units = 'in', res = 600)
ggplot(pcaR, aes(PC1, PC2, color= prognosis, shape=Sex)) + geom_point(alpha=0.6,stroke = 3)+
  xlab(paste0("PC1: ",percentVar[1],"% variance")) +
  ylab(paste0("PC2: ",percentVar[2],"% variance")) +
  coord_fixed()  +theme_bw()
#dev.off ()

In [29]:
#----- If By anyMethod - like PCA, Variance partition, differential Expression You have found few Genes which are Significantly Discriminating
#---- Can you pick Those Genes and Make a HeatMap of Those selected Genes""
Sel_genes <- c('UGT2B17', 'GSTM1', 'XRRA1', 'AKR1B15', 'DDTL', 'GBP3', 'DSG3', 'CYP4F11', 'DMRTA1', 'ERAP2', 'MYADML2', 'GRSF1', 'HOXA13', 'BTN3A2', 'UGT3A1', 'GSTM3', 'ARL17B', "IL26", "HLA.G")
# In this file VSD nomralized Counts were there head (Gene_counts_Factor_vsd_df_t)
Sel_genes_vsd <- Gene_counts_Factor_vsd_df_t[Sel_genes]
str (Sel_genes_vsd)

In [30]:
#- Again check the row.names in metadata_final and this file
row.names (metadata_filtered)
row.names (Sel_genes_vsd)

In [31]:
colors <- colorRampPalette( rev(brewer.pal(9, "Blues")) )(255)

#jpeg("Figures/SelectedGenes_ResidualsModelA_HeatMap.jpg", height =11, width = 11, units = 'in', res = 600)
check <- pheatmap(Sel_genes_vsd, scale = "column",
                  col=colors, annotation_row= metadata_filtered, show_rownames =FALSE,show_colnames = TRUE,legend = TRUE,legend.cex = .05)
#dev.off ()

ERROR: Error in parse(text = x, srcfile = src): <text>:5:29: unexpected ','
4: #check <- pheatmap(Sel_genes_vsd, scale = "column",
5:                   col=colors,
                               ^


In [ ]:
#---- Now you need to Select the Genes based on pvlues, fold changes of variance partitions.
#-- There are Many more things which can be done.
#-- Sel around 600 Genes from Results_DifferentialExpression.txt file Based on p-values
# Important genes can be used to do pathway analysis - metascape
#https://metascape.org/gp/index.html#/main/step1

In [18]:
#-- If you are Interested and would like to use Machine Learning on Expression data:
#-- If ChemBioIT will organize furhter courses then I would be happy to deliver this lecture.

In [22]:
#BiocManager::install("org.Hs.eg.db")
library("AnnotationDbi")
library("org.Hs.eg.db")
columns(org.Hs.eg.db)

[1] "ACCNUM"       "ALIAS"        "ENSEMBL"      "ENSEMBLPROT"  "ENSEMBLTRANS"
 [6] "ENTREZID"     "ENZYME"       "EVIDENCE"     "EVIDENCEALL"  "GENENAME"    
[11] "GO"           "GOALL"        "IPI"          "MAP"          "OMIM"        
[16] "ONTOLOGY"     "ONTOLOGYALL"  "PATH"         "PFAM"         "PMID"        
[21] "PROSITE"      "REFSEQ"       "SYMBOL"       "UCSCKG"       "UNIGENE"     
[26] "UNIPROT"

In [ ]:
# Follow this link - If teaching this part in R
# https://www.r-bloggers.com/2015/12/tutorial-rna-seq-differential-expression-pathway-analysis-with-sailfish-deseq2-gage-and-pathview/
res$symbol = mapIds(org.Hs.eg.db,
                     keys=row.names(res), 
                     column="SYMBOL",
                     keytype="ENSEMBL",
                     multiVals="first")